<a href="https://colab.research.google.com/github/Vinisenso/Lotsizing_problem_Textile_Factory/blob/main/LE505_Trabalho_Final_Grupo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grupo 2:

- Augusto Chebel Machado  RA: 231762

- João Pedro Marretto Helmeister RA: 174564

- Lucas Massabni Busnardo RA: 182516

- Matheus Rodrigues  RA: 185315

- Nathan Czelusniak de Siqueira RA: 186041
 
- Vinícius Consentino Muniz RA: 245271



In [ ]:
# Instalando duas bibliotecas: ortools serve para usar ferramentas de PO e gspread para importar os dados do Google Sheets.

!pip install ortools --quiet
!pip install gspread

In [ ]:
#Código que importa dados do Google Sheets

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
#Importando outras bibliotecas importantes para o problema.

import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from google.colab import files
import matplotlib.pyplot as plt



In [ ]:
# Importando os nossos dados do problema

worksheet = gc.open('ER460 - Matriz de Decisão (Escolha de um Automóvel)').sheet1

# Fazendo um Data Frame dos nossos dados. # Mostra a planilha com os dados completos (nome+valores)
df = pd.DataFrame(worksheet.get_all_records())
df

NameError: ignored

In [ ]:
# Importanto dados de um arquivo excel.

#Professor, caso seja necessário importar os dados de um arquivo excel, você pode tirar as "#" do código abaixo. Além disso, faça o upload do arquivo excel na aba no canto esquerdo da página, onde tem um ícone de documento, por favor.

# df0=pd.read_excel("Dados Trabalho de PO Completo 2022.xlsx", index_col = "xit")
# df0

In [ ]:
#Tirando a coluna 1 do nosso data frame, a qual possuia o nome das peças. Queremos trabalhar apenas com os dados numéricos a partir de agora.

df=df0.drop(columns="xit")
df

In [ ]:
# Indentificando colunas importantes para a resolução do trabalho com os nomes técnicos, os quais vão ser usados ao longo do código.

columns = df.columns

#coluna

nome_fi = "fi"
nome_vcit = "vcit"
nome_vtit = "vtit"
nome_stit = "stit"
nome_hcit = "hcit"
nome_scit = "scit"


fi = columns.get_loc(nome_fi)
vcit = columns.get_loc(nome_vcit)
vtit = columns.get_loc(nome_vtit)
stit = columns.get_loc(nome_stit)
hcit = columns.get_loc(nome_hcit)
scit = columns.get_loc(nome_scit)

#column_index

print("Index da coluna", nome_fi, "é" , fi)
print("Index da coluna", nome_vcit, "é" , vcit)
print("Index da coluna", nome_vtit, "é" , vtit)
print("Index da coluna", nome_stit, "é" , stit)
print("Index da coluna", nome_scit, "é" , scit)
print("Index da coluna", nome_hcit, "é" , hcit)


In [ ]:
#Transformando os dados em um Array
matriz=df.to_numpy()


In [ ]:
#Criando um problema em PO
solver = pywraplp.Solver.CreateSolver('SCIP') # O SCIP é o Solver que resolve o problema inteiro.

In [ ]:
# Adicionando as variáveis

maq = 2

x= {(i,k,t):solver.IntVar(lb=0,ub=solver.infinity(),name=f'x_{i+1}_{k+1}_{t+1}') # xikt: quantidade produzida de cada produto i na máquina k no período t
      for i in range(len(matriz))
      for k in range(0,maq)
      for t in range(0,12)}
print(x)

i=0
h = {(i,t):solver.IntVar(lb=0,ub=solver.infinity(),name=f'h_{i+1}_{t}') # hit: quantidade estocada de cada produto i no período t.
      for i in range(len(matriz))
      for t in range(0,13)}
print(h)

i=0
y= {(i,k,t):solver.BoolVar(name=f'y_{i+1}_{k+1}_{t+1}')  # yikt: variável binária indicando se foi ou não produzido o produto i na máquina k no período t.
        for i in range(len(matriz))
        for k in range(0,maq)
        for t in range(0,12)}
print(y)

In [ ]:
# Adicionando a função objetivo do problema.

solver.Maximize(sum(x[(i,k,t)]*(matriz[i][fi]-matriz[i][vcit]) for i in range(0,len(matriz)) for k in range (0,maq) for t in range(0,12))-
                sum(h[(i,t)]*(matriz[i][hcit]) for i in range(0,len(matriz)) for t in range(0,13))-
                sum(y[(i,k,t)]*(matriz[i][scit]) for i in range(0,len(matriz)) for k in range (0,maq) for t in range(0,12)))
print(solver.ExportModelAsLpFormat(obfuscated=False))

In [ ]:
#Adicionando as restrições de estoque
for i in range(len(matriz)):
  for t in range(0,12):
    solver.Add(sum(x[(i,k,t)] for k in range (0,maq)) + h[(i,t)] == matriz[i][t] + h[(i,t+1)])

In [ ]:
#Adicionando a restrição que não posso produzir mais do que preciso
for i in range(len(matriz)):
  for k in range(0, maq):
    for t in range(0,12):
      solver.Add(x[(i,k,t)] <= sum(matriz[i][t:12])*y[(i,k,t)])

In [ ]:
# Adicionando a restrição da disponibilidade de tempo

#Quem quiser pode variar as horas e os dias na semana que a empresa trabalha para avaliar cenários variados

min=60 # Quantos minutos tem 1 hora.
ho=24 # Horas no dia que a máquina trabalha
d=24  # Dias no mês que a máquina trabalha

Cap=d*ho*min 
print(f'O total de minutos que a máquina vai operar neste dia é {Cap} minutos.')

for t in range(0,12):
  for k in range(0,maq):
    solver.Add(sum(x[(i,k,t)]*matriz[i][vtit] for i in range(0,len(matriz)))+ sum(y[(i,k,t)]*matriz[i][stit] for i in range(0,len(matriz)))<=Cap)

In [ ]:
#Zerando o estoque inicial e final pois se trata de um problema com horizonte temporal finito

t_estoque_0=[0,12]

for i in range(0,len(matriz)):
  for t in t_estoque_0:
    solver.Add(h[(i,t)]==0)

**Estruturação:**

In [ ]:
# Mostra a formulação final do problema com a quantidade de variáveis e restrições.

print(solver.ExportModelAsLpFormat(obfuscated=False))

**Resultados:**

In [ ]:
#Pedindo para o Solver resolver o problema.
solver.Solve();

In [ ]:
# Printando o resultado do problema

obj=solver.Objective().Value()
print(f'A empresa teve um lucro maximizado de {obj} reais\n')
c=0
d=0
f=0
#Printando os valores das variáveis diferentes de 0.
print('\nProdutos produzidos em cada período e máquina e sua respectiva quantidade:\n')
for i in x.values():
  if i.solution_value() > 0:
    print(i,'=',i.solution_value())
    c+=1
print(f'\nHouve {c} processos produtivos no ano, calculado a partir da variável xit.\n')  


print('\nProdutos estocados em cada período e sua respectiva quantidade:\n')
for i in h.values():
  if i.solution_value() > 0:
    print(i,'=',i.solution_value())
    d+=1
print(f'\nHouve {d} processos de estoque no ano.\n')  

print('\n Períodos e máquinas em que cada produto foi produzido:\n')
for i in y.values():
  if i.solution_value() > 0:
    print(i,'=',i.solution_value())
    f+=1
print(f'\nHouve {f} processos produtivos no ano, calculados pela variável yikt, valor consoante ao calculado a partir do xikt.\n')      







In [ ]:
#Parâmetros de desempenho do modelo:

print(f'Os parâmetros de desempenho do modelo com {maq} máquina(s) são:')
print('\nO problema resolvido em %d iterações.' % solver.iterations())
print(f'\nO tempo computacional exigido para o código é: {round(solver.wall_time()/1000,2)} s.')
print(f'\nO número de nós exigidos no método Branch and Bound é {solver.nodes()} nós.')


**Tratamento e Análise dos Resultados**

In [ ]:
# Tabela bem simplificada com os valores das variáveis
# Pode ser usada para obtenção de informações - célula abaixo permite baixar no formato excel

dados=list()

for i in x.values():
  # print(i,'=',i.solution_value())
  dados.append([i,i.solution_value()])
for i in h.values():
  dados.append([i,i.solution_value()]) 
for i in y.values():
   dados.append([i,i.solution_value()])
print(dados) 

dff=pd.DataFrame(dados,columns=["Produto","Quantidade"])
dff

In [ ]:
#Baixando a tabela acima para excel.
dff.to_excel("ResultadosPO.xlsx",sheet_name="ResultadosPO",index=False,header=True,na_rep=" ")

In [ ]:

# Alguns testes são feitos para validar se as restrições foram respeitadas

#Respeitou o tempo diponível?
tempo=np.arange(1,13,1)
ltempo=np.zeros(12)
ltempot=np.zeros(12)

# tratando das máquinas juntas 
for j in range(0,12):
  ltempot[j]=(sum(x[(i,k,j)].solution_value()*matriz[i][vtit] + y[(i,k,j)].solution_value()*matriz[i][stit] for i in range(0,34) for k in range(0,maq)))
print(ltempot)


for k in range(0,maq):
  for j in range(0,12):
    ltempo[j]=(sum(x[(i,k,j)].solution_value()*matriz[i][vtit] + y[(i,k,j)].solution_value()*matriz[i][stit] for i in range(0,34)))
  print(ltempo)

 


In [ ]:
# Plotando um gráfico com os tempos de produção utilizados em cada perído

plt.figure(figsize=[20,7])
plt.grid()
plt.title("Capacidade temporal utilizada por período.", fontsize=16)
plt.xlabel("Mês",labelpad=10,fontsize=16)
plt.ylabel("Tempo (min)",fontsize=16)
plt.bar(tempo,ltempot,edgecolor='black',tick_label=tempo,width=0.75)
plt.hlines(maq*Cap,0,13.4,colors="r",label=f"Tempo \n total: \n {maq*Cap} min")
plt.legend(loc="lower right", fontsize=12)

for i in range(0,12):
  plt.text(i+0.75, ltempot[i]-4000, str(int(ltempot[i])),fontsize=12,bbox=dict(facecolor='grey',alpha=0.8))

plt.show()

In [ ]:
ltempo=np.zeros(12)
for k in range(0,maq):
  for j in range(0,12):
    ltempo[j]=(sum(x[(i,k,j)].solution_value()*matriz[i][vtit] + y[(i,k,j)].solution_value()*matriz[i][stit] for i in range(0,34)))
  plt.figure(figsize=[20,7])  
  plt.grid()
  plt.title(f"Capacidade temporal utilizada pela máquina {k+1} por período.", fontsize=16)
  plt.xlabel("Mês",labelpad=10,fontsize=16)
  plt.ylabel("Tempo (min)",fontsize=16)
  plt.bar(tempo,ltempo,edgecolor='black',tick_label=tempo,width=0.75)
  plt.scatter(0,37000,s=0.001)
  plt.hlines(Cap,0,13.4,colors="r",label=f"Tempo \n total: \n {Cap} min")
  plt.legend(loc="lower right", fontsize=12)

  for i in range(0,12):
    plt.text(i+0.75, ltempo[i]+900, str(int(ltempo[i])),fontsize=12,bbox=dict(facecolor='grey',alpha=1))

  plt.show()

In [ ]:
 
# cores={1:"b",2:"g"}
ltempo1=np.zeros(12)
ltempo2=np.zeros(12) 
if maq==2:
  for j in range(0,12):
    ltempo1[j]=(sum(x[(i,0,j)].solution_value()*matriz[i][vtit] + y[(i,0,j)].solution_value()*matriz[i][stit] for i in range(0,34)))
    ltempo2[j]=(sum(x[(i,1,j)].solution_value()*matriz[i][vtit] + y[(i,1,j)].solution_value()*matriz[i][stit] for i in range(0,34)))

  plt.figure(figsize=[20,7])
  plt.grid()
  plt.title(f"Comparação capacidade utilizada por cada máquina por período.", fontsize=16)
  plt.xlabel("Mês",labelpad=10,fontsize=16)
  plt.ylabel("Tempo (min)",fontsize=16)
  plt.scatter(0,37000,s=0.001)  
  plt.scatter(14,37000,s=0.001)  
  plt.hlines(Cap,0,13.4,colors="r",label=f"Tempo \n total: \n {Cap} min")

  
    
  plt.bar(tempo,ltempo1,edgecolor='black',tick_label=tempo,width=0.4,color="g",label="Máquina 1")
  plt.bar(tempo+0.5,ltempo2,edgecolor='black',tick_label=tempo,width=0.4,color="b",label="Máquina 2")
  for i in range(0,12):
      plt.text(i+0.75,ltempo1[i]+900, str(int(ltempo1[i])),fontsize=9,bbox=dict(facecolor='grey',alpha=1))
      plt.text(i+1.3, ltempo2[i]+1000, str(int(ltempo2[i])),fontsize=9,bbox=dict(facecolor='grey',alpha=1))
  
  plt.legend(loc="lower right", fontsize=12)
  plt.show()

In [ ]:
#Estoque total em cada mes:
tempo2=np.arange(0,13,1)
lh=np.zeros(13)

for j in range(0,13):
  lh[j]=sum(h[(i,j)].solution_value() for i in range(0,len(matriz)))
print(lh)

In [ ]:
#Plotando o estoque total de cada mês.
if maq==1:  
  plt.figure(figsize=[20,7])
  plt.grid()
  plt.title("Estoque utilizado por período.", fontsize=16)
  plt.xlabel("Mês",labelpad=5)
  plt.ylabel("Quantidade")
  plt.bar(tempo2,lh,edgecolor='black',tick_label=tempo2)

  plt.text(-0.5,850,f'Custo total de estoque: R${sum(lh)*0.5}',fontsize=16,bbox=dict(facecolor='grey',alpha=0.5))
  plt.text(-0.5,950,f'Quantidade total de estoque: {sum(lh)}',fontsize=16,bbox=dict(facecolor='grey',alpha=0.5))

  for i in range(0,12):
    plt.text(i-0.25, lh[i] + 20, str(lh[i]),fontsize=11,bbox=dict(facecolor='grey',alpha=0.5))

  plt.show()
else:  
  plt.figure(figsize=[20,7])
  plt.grid()
  plt.title("Estoque utilizado por período.",fontsize=16)
  plt.xlabel("Mês",labelpad=5,fontsize=14)
  plt.ylabel("Quantidade",fontsize=14)
  plt.bar(tempo2,lh,edgecolor='black',tick_label=tempo2)

  plt.text(6,260+20,f'Custo total de estoque: R${sum(lh)*0.5}',fontsize=16,bbox=dict(facecolor='grey',alpha=0.5))
  plt.text(6,240+20,f'Quantidade total de estoque: {sum(lh)}',fontsize=16,bbox=dict(facecolor='grey',alpha=0.5))

  for i in range(0,13):
    plt.text(i-0.25, lh[i] + 5, str(lh[i]),fontsize=11,bbox=dict(facecolor='grey',alpha=0.5))

  plt.show()  

In [ ]:
#Quantidade total produzida em cada mês:
tempo=np.arange(1,13,1)
lp=np.zeros(12)

for j in range(0,12):
  lp[j]=sum(x[(i,k,j)].solution_value() for i in range(0,34) for k in range(0,maq))
print()

In [ ]:
#Plotando a quantidade produzida em cada mês.
plt.figure(figsize=[20,7])
plt.grid()
plt.title("Produção por período.",fontsize=16)
plt.xlabel("Mês",labelpad=5,fontsize=14)
plt.ylabel("Quantidade",fontsize=14)
plt.bar(tempo,lp,edgecolor='black',tick_label=tempo)

demanda=0
demandat=list()
for i in range(0,34):
  for j in range(0,12):
    demanda=0+matriz[i][j]
    demandat.append(demanda)

plt.text(0.25,2200,f'Total Produzido: {sum(lp)}',fontsize=16,bbox=dict(facecolor='grey',alpha=0.2))
plt.text(0.25,2000,f'Soma da Demanda: {sum(demandat)}',fontsize=16,bbox=dict(facecolor='grey',alpha=0.2))
for i in range(0,12):
  plt.text(i+0.8, lp[i] + 50, str(int(lp[i])),fontsize=11,bbox=dict(facecolor='grey',alpha=0.5))
plt.show()



In [ ]:
lp1=np.zeros(12)
lp2=np.zeros(12)
if maq==2:
  for j in range(0,12):
    lp1[j]=sum(x[(i,0,j)].solution_value() for i in range(0,34))
    lp2[j]=sum(x[(i,1,j)].solution_value() for i in range(0,34))

  plt.figure(figsize=[20,7])
  plt.grid()
  plt.title(f"Total produzido por cada máquina por período.", fontsize=16)
  plt.xlabel("Mês",labelpad=10,fontsize=16)
  plt.ylabel("Tempo (min)",fontsize=16)
  plt.scatter(14,2500,s=0.001)  
  plt.scatter(14,2000,s=0.001)  


  
    
  plt.bar(tempo,lp1,edgecolor='black',tick_label=tempo,width=0.4,color="g",label="Máquina 1")
  plt.bar(tempo+0.5,lp2,edgecolor='black',tick_label=tempo,width=0.4,color="b",label="Máquina 2")
  for i in range(0,12):
      plt.text(i+0.9,lp1[i]+55, str(int(lp1[i])),fontsize=9,bbox=dict(facecolor='grey',alpha=1))
      plt.text(i+1.4, lp2[i]+55, str(int(lp2[i])),fontsize=9,bbox=dict(facecolor='grey',alpha=1))
  plt.text(1,2250, f"Total Produzido Máquina 1 {str(int(sum(lp1)))}",fontsize=14,bbox=dict(facecolor='g',alpha=0.5))
  plt.text(1,2050 ,f"Total Produzido Máquina 2 {str(int(sum(lp2)))}",fontsize=14,bbox=dict(facecolor='b',alpha=0.5)) 
  plt.legend(loc="lower right", fontsize=12)
  plt.show()